# Module 6: Mid-Line Detection

This is where we start to get to the main application for our Jetbot. We want to be able to detect the lane and find the trajectory for which our robot should follow. This will involve some of the lessons we have previously learned, and applying them in a separate context.

This module should follow Module 5: Road Sign Detection

Below is our normal starter code for initializing all variables and packages.

In [ ]:
from jetbot import Robot, Camera, bgr8_to_jpeg
import cv2
import numpy as np
import ipywidgets as widgets
from scipy.signal import medfilt
from IPython.display import display, Image, clear_output
import time
import threading
import os

## Method

Normally, for a regular autonomous vehicle, the car is programmed to identify the left and right lane markings, then develop the path and trajectory that the car should take. But this often involves multiple sensors and cameras that have both a wide field-of-view, as well as at various locations. But our Jetbot only has a singular camera, and as you may have seen in previous modules, the field-of-view is not particularly large. Thus, we are attempting a modified version, where we will detect the yellow middle markings, and have the Jetbot simply follow that dotted line.

## Implementation

To accomplish our task, we will undergo the following steps to our image.

1. HSV Color Masking

    One of the many ways to differentiate the yellow dotted line from the rest of the image, is to isolate the color yellow. To do this, we will use HSV color masking.

2. Sobel Edge Detection

    After we have isolated only the color yellow, we want to detect the edges, which highlight major changes in values between pixels. This also involves many other pre-processing techniques, including blurring and filtering based on area and direction.
    
    Sobel Edge detection also primarily relies on directionality when determining edges. This means we can isolate the vertical edges in particular, which will be useful for our line tracking.

3. Midpoint Calculation

    The Sobel Edge Detection will actually detect two vertical edges on each side of the yellow lane marker. But for our Jetbot, we want it to follow in the middle of these two points. This process is already completed, but it also involves filtering outliers and major deviations.

### Activity

Similar to Module 5, we are going to take photos with our <i><b>Camera_Capture.ipynb</i></b> notebook of various angles and positions of the yellow lane marking. Once you do that, run the following code below to confirm that the images did save.

In [ ]:
# TODO: Change folder name HERE!!!
SAVE_DIR = "lane_images"
# Optional: Change filename here to look at different image.
filename = "image_0.jpg"
# Displays the image
filepath = os.path.join(SAVE_DIR, filename)
frame = cv2.imread(filepath)
display(Image(data=bgr8_to_jpeg(frame)))

This next block has many of the functions that we will use to detect our yellow center line. When you run the code below, a few widgets should appear below. These widgets will allow you to modify various values and parameters, which you can tune to find the best values for detecting the yellow lane markings.

In [ ]:
# Brighten Gamma of Image
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Filter based on inter-quartile range (remove outliers)
def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

# Calculate midpoints based on masked image
def lane_midpoint(masked_image):
    midline_points = []
    left_points = []
    right_points = []
    h, w = masked_image.shape[:2]

    for y in range(0, h, 4):
        x_candidates = [x for x in range(w) if masked_image[y, x] > 0]
        if len(x_candidates) >= 3:
            x_candidates = iqr_filter(x_candidates)
            left_x = min(x_candidates)
            right_x = max(x_candidates)
            center_x = int(np.mean(x_candidates))

            midline_points.append((center_x, y))
            left_points.append((left_x, y))
            right_points.append((right_x, y))

    if midline_points:
        x_vals = [x for x, _ in midline_points]
        x_smooth = medfilt(x_vals, kernel_size=5)
        midline_points = list(zip(x_smooth, [y for _, y in midline_points]))
    
    return midline_points, left_points, right_points

# Isolate Yellow Lane Marking
def get_yellow_lane(image, apply_gamma=True):

    # Brighten Images
    if apply_gamma:
        image = adjust_gamma(image, gamma=1.2)

    # Blur and Convert to HSV Color Space
    blur = cv2.bilateralFilter(image, 9, 75, 75)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Read HSV range from sliders
    lower_yellow = (h_min.value, s_min.value, v_min.value)
    upper_yellow = (h_max.value, s_max.value, v_max.value)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Morphology to clean specks (Dilation and Erosion)
    kernel = np.ones((5, 5), np.uint8)
    opened = cv2.morphologyEx(mask_yellow, cv2.MORPH_OPEN, kernel, iterations=1)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Remove small blobs
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    clean_mask = np.zeros_like(closed)
    for cnt in contours:
        if cv2.contourArea(cnt) > 150:
            cv2.drawContours(clean_mask, [cnt], -1, 255, thickness=cv2.FILLED)
    contour_mask = clean_mask

    # Edge detection
    yellow_mask_blur = cv2.GaussianBlur(contour_mask, (5, 5), 0)
    sobelx = cv2.Sobel(yellow_mask_blur, cv2.CV_64F, 1, 0, ksize=3)
    sobelx = np.absolute(sobelx)
    if np.max(sobelx) > 0:
        sobelx = np.uint8(255 * sobelx / np.max(sobelx))
    else:
        sobelx = np.zeros_like(sobelx, dtype=np.uint8)
    _, edge_mask = cv2.threshold(sobelx, 50, 255, cv2.THRESH_BINARY)

    return mask_yellow, edge_mask


# Updating Loop
stop_event = threading.Event()

def update_loop():
    while not stop_event.is_set():
        image_capture = frame
        
        yellow_mask, edge_mask = get_yellow_lane(frame)

        midline_points, left_points, right_points = lane_midpoint(edge_mask)

        overlay = frame.copy()
        for (x, y) in midline_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)
        for (x, y) in left_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)
        for (x, y) in right_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)

        # Ensure UI-safe conversions
        overlay_widget.value = bgr8_to_jpeg(overlay)
        yellow_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(yellow_mask, cv2.COLOR_GRAY2BGR))
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edge_mask, cv2.COLOR_GRAY2BGR))

        time.sleep(0.1)

# Yellow HSV Slider Widgets
h_min = widgets.IntSlider(value=0, min=0, max=180, description='H Min')
s_min = widgets.IntSlider(value=0, min=0, max=255, description='S Min')
v_min = widgets.IntSlider(value=0, min=0, max=255, description='V Min')
h_max = widgets.IntSlider(value=180, min=0, max=180, description='H Max')
s_max = widgets.IntSlider(value=255, min=0, max=255, description='S Max')
v_max = widgets.IntSlider(value=255, min=0, max=255, description='V Max')

display(widgets.VBox([
    widgets.Label("Adjust HSV range for yellow detection:"),
    widgets.HBox([h_min, h_max]),
    widgets.HBox([s_min, s_max]),
    widgets.HBox([v_min, v_max])
]))

# Image Debugging Widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
yellow_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([
    widgets.VBox([widgets.Label("Overlay with midline"), overlay_widget]),
    widgets.VBox([widgets.Label("Yellow Mask"), yellow_mask_widget]),
    widgets.VBox([widgets.Label("Edge Mask"), edge_mask_widget])
]))

# Starts the Feed
thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()

## Live Camera Testing

Once you find values and tracking that you like, try it with the live camera feed from the <b><span style="color:#154734">JetBot</span></b>.

You will need to update any values you changed above. These do not save between blocks.

In [ ]:
# Initialize our robot
robot = Robot()
# Initialize our camera
camera = Camera.instance()

# Save image data (modified to be numpy array)
image = np.array(camera.value)

jpeg_image = bgr8_to_jpeg(image)

# Display image within Jupyter Notebook
display(Image(data=jpeg_image))

In [ ]:
# Updating Loop
stop_event = threading.Event()

def update_loop():
    while not stop_event.is_set():
        frame = camera.value
        
        yellow_mask, edge_mask = get_yellow_lane(frame)

        midline_points, left_points, right_points = lane_midpoint(edge_mask)

        overlay = frame.copy()
        for (x, y) in midline_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)
        for (x, y) in left_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)
        for (x, y) in right_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)

        # Ensure UI-safe conversions
        overlay_widget.value = bgr8_to_jpeg(overlay)
        yellow_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(yellow_mask, cv2.COLOR_GRAY2BGR))
        edge_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(edge_mask, cv2.COLOR_GRAY2BGR))

        time.sleep(0.1)

# Yellow HSV Slider Widgets
h_min = widgets.IntSlider(value=0, min=0, max=180, description='H Min')
s_min = widgets.IntSlider(value=0, min=0, max=255, description='S Min')
v_min = widgets.IntSlider(value=0, min=0, max=255, description='V Min')
h_max = widgets.IntSlider(value=180, min=0, max=180, description='H Max')
s_max = widgets.IntSlider(value=255, min=0, max=255, description='S Max')
v_max = widgets.IntSlider(value=255, min=0, max=255, description='V Max')

display(widgets.VBox([
    widgets.Label("Adjust HSV range for yellow detection:"),
    widgets.HBox([h_min, h_max]),
    widgets.HBox([s_min, s_max]),
    widgets.HBox([v_min, v_max])
]))

# Image Debugging Widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
yellow_mask_widget = widgets.Image(format='jpeg', width=300, height=300)
edge_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([
    widgets.VBox([widgets.Label("Overlay with midline"), overlay_widget]),
    widgets.VBox([widgets.Label("Yellow Mask"), yellow_mask_widget]),
    widgets.VBox([widgets.Label("Edge Mask"), edge_mask_widget])
]))

# Starts the Feed
camera.start()
thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

Make sure to move the <b><span style="color:#154734">JetBot</span></b> around to ensure the camera sees the yellow lane markings at various angles and locations!!! Don't just test the camera in one location!

The code below allows you to break the loop in case you want to edit anything in the function. Run the following block, edit anything you need to above, then run the blocks from that point downwards.

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()

## Bonus Challenge

If you want an extra challenge, see if you can modify the code above to detect the left and right lane and find the midpoint trajectory from those lanes!

In [ ]:
# Image Debugging Widgets
overlay_widget = widgets.Image(format='jpeg', width=300, height=300)
lane_mask_widget = widgets.Image(format='jpeg', width=300, height=300)

display(widgets.HBox([
    widgets.VBox([widgets.Label("Overlay with midline"), overlay_widget]),
    widgets.VBox([widgets.Label("Lane Mask"), lane_mask_widget])
]))

# Brighten Gamma of Image
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([(i / 255.0) ** invGamma * 255 for i in np.arange(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Filter based on inter-quartile range (remove outliers)
def iqr_filter(x_list):
    x = np.array(x_list)
    q1, q3 = np.percentile(x, [25, 75])
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    return [val for val in x if lower <= val <= upper]

# Calculate midpoints based on masked image
def lane_midpoint(masked_image):
    midline_points = []
    left_points = []
    right_points = []
    h, w = masked_image.shape[:2]

    for y in range(0, h, 4):
        x_candidates = [x for x in range(w) if masked_image[y, x] > 0]
        if len(x_candidates) >= 3:
            x_candidates = iqr_filter(x_candidates)
            left_x = min(x_candidates)
            right_x = max(x_candidates)
            center_x = int(np.mean(x_candidates))

            midline_points.append((center_x, y))
            left_points.append((left_x, y))
            right_points.append((right_x, y))

    if midline_points:
        x_vals = [x for x, _ in midline_points]
        x_smooth = medfilt(x_vals, kernel_size=5)
        midline_points = list(zip(x_smooth, [y for _, y in midline_points]))
    
    return midline_points, left_points, right_points

# Isolate Yellow Lane Marking
def get_lane_markings(image, apply_gamma=True):
    # TODO: Add Code Here!
    # Can add more returns for debugging
    return lane_mask

stop_event = threading.Event()

def update_loop():
    while not stop_event.is_set():
        frame = camera.value

        # Can add more returns for debugging.
        lane_mask = get_lane_markings(frame)

        midline_points, left_points, right_points = lane_midpoint(lane_mask)

        overlay = frame.copy()
        for (x, y) in midline_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 255, 0), -1)
        for (x, y) in left_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (255, 0, 0), -1)
        for (x, y) in right_points:
            cv2.circle(overlay, (int(x), int(y)), 2, (0, 0, 255), -1)

        # Ensure UI-safe conversions
        overlay_widget.value = bgr8_to_jpeg(overlay)
        lane_mask_widget.value = bgr8_to_jpeg(cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR))

        time.sleep(0.1)

camera.start()
thread = threading.Thread(target=update_loop, daemon=True)
thread.start()

In [ ]:
stop_event.set()
thread.join()
stop_event.clear()
camera.stop()